In [1]:
import pandas as pd

df_lifesat = pd.read_csv("./data/lifesat_full.csv")
df_lifesat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36 entries, 0 to 35
Data columns (total 6 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Country               36 non-null     object 
 1   GDP per capita (USD)  36 non-null     float64
 2   Life satisfaction     36 non-null     float64
 3   Life expectancy       36 non-null     float64
 4   Homicide rate         36 non-null     float64
 5   Employment rate       36 non-null     float64
dtypes: float64(5), object(1)
memory usage: 1.8+ KB


## Función describe_df

In [2]:
def describe_df(df):
    '''
    Descripción: Para cada columna del dataframe, devuelve el tipo de variable, el
    porcentaje de nulos, los valores únicos y porcentaje de cardinalidad.

    Argumentos:
    df (dataframe)

    Retorna:
    dataframe: dataframe con las columnas del dataframe original, y con las filas
    de los tipos de variables, el tanto por ciento de nulos, los valores únicos
    y el porcentaje de cardinalidad.
    '''

    # Data_type
    data_type = df.dtypes

    # Nulos
    nulos = df.isna().sum()
    porcentaje_nulos = (nulos / len(df) * 100).round(2)

    # Valores únicos
    unicos = df.nunique()

    # Cardinalidad
    card = (unicos / len(df) * 100).round(2)

    # Creación tabla
    tabla = pd.DataFrame({
        "DATA_TYPE": data_type,
        "MISSINGS (%)": porcentaje_nulos,
        "UNIQUE_VALUES": unicos,
        "CARDIN (%)": card
    },
    index = pd.Index(df.columns, name = "COL_N"))

    return tabla.T # devuelve la traspuesta

## Función tipifica_variables

In [10]:
def tipifica_variables(df, umbral_categoria, umbral_continua):
    '''
    Clasifica las variables del dataframe df en Binaria, Categórica, Numérica continua
    o Numérica Discreta, según el siguiente criterio:
    - Si la cardinalidad es 2, asignara "Binaria"
    - Si la cardinalidad es menor que `umbral_categoria` asignara "Categórica"
    - Si la cardinalidad es mayor o igual que `umbral_categoria`, entonces entra en juego el tercer argumento:
        * Si además el porcentaje de cardinalidad es superior o igual a `umbral_continua`, asigna "Numerica Continua"
        * En caso contrario, asigna "Numerica Discreta"

    Argumentos:
    df (dataframe)
    umbral_categoria (entero): para determinar si una variable es categórica
    umbral_continua (float): para determinar si una variable numérica es continua o discreta

    Retorna:
    dataframe: dataframe con dos columnas "nombre_variable" y "tipo_sugerido".
    '''

    # 

    df_descripcion = describe_df(df).T # aprovechamos la función describe_df que ya calcula la cardinalidad y hacemos la transpuesta, para que los nombres de
                                     # las variables sean las filas y no las columnas

    # Corrección para cuando solo tengo un valor
    df_descripcion.loc[df_descripcion["UNIQUE_VALUES"] == 1, "CARDIN (%)"] = 0.00

    # Creamos una nueva columna "tipo_sugerido", partiendo de que todas son categóricas por ejemplo
    df_descripcion["tipo_sugerido"] = "Categórica"

    # Clasificamos
    df_descripcion.loc[df_descripcion["UNIQUE_VALUES"] == 2, "tipo_sugerido"] = "Binaria"
    df_descripcion.loc[df_descripcion["UNIQUE_VALUES"] >= umbral_categoria, "tipo_sugerido"] = "Numérica discreta"
    df_descripcion.loc[df_descripcion["CARDIN (%)"] >= umbral_continua, "tipo_sugerido"] = "Numérica continua"

    # Reseteamos el index para que pase a ser la columna "nombre_variable"
    df_descripcion = df_descripcion.reset_index(names = ["nombre_variable"])

    # Eliminamos columnas dejando solo "nombre_variable" y "tipo_sugerido"
    df_descripcion = df_descripcion.drop(columns = ["DATA_TYPE", "MISSINGS (%)", "UNIQUE_VALUES", "CARDIN (%)"])

    return df_descripcion

# Función get_features_num_regression

In [ ]:
def get_features_num_regression(df, target_col, umbral_corr, pvalue = None):
    '''
    Descripción: 

    Argumentos: 
    df (dataframe)
    target_col (columna): columna de df, que debería ser el target de un hipotético modelo de regresión. Es una variable numérica con alta cardinalidad.
    umbral_corr (float): debe estar entre 0 y 1.
    pvalue (float, por defecto None): 

    Retorna:

    '''

# Función plot_features_num_regression

In [ ]:
def plot_features_num_regression(df, target_col, columns = [], umbral_corr = 0, pvalue = None):
    '''
    Descripción: pinta un pairplot

    Argumentos:
    df(dataframe)
    target_col (columna):
    columns (lista, por defecto vacía):
    umbral_corr (float, por defecto 0):
    pvalue (float, por defecto None):

    Retorna:

    '''

# Función get_features_cat_regression

In [ ]:
def get_features_cat_regression(df, target_col, pvalue = 0.05):
    '''
    Descripción:

    Argumentos:
    df (dataframe)
    target_col (columna):
    pvalue (float):

    Retorna:
    
    '''

# Función plot_features_cat_regression

In [ ]:
def plot_features_cat_regression(df, target_col = "", columns = [], pvalue = 0.05, with_individual_plot = False):
    '''
    Descripción:

    Argumentos:
    df(dataframe)
    target_col (columna):
    columns (lista, por defecto vacía):
    pvalue (float, por defecto None):
    with_individual_plot (booleano):

    Retorna:
    
    '''